In [67]:
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import NearestNeighbors
%matplotlib inline

In [68]:
DEV_MODE = False

In [69]:
def load_and_preprocess_data():
    # Load the datasets
    df_net = pd.read_csv('data/linux_driver_net.csv')
    df_scsi = pd.read_csv('data/linux_driver_scsi.csv')
    df_ext3 = pd.read_csv('data/linux_ext3.csv')
    df_ipv4 = pd.read_csv('data/linux_ipv4.csv')
    
    # Combine datasets
    combined_df = pd.concat([df_net, df_scsi, df_ext3, df_ipv4], ignore_index=True)
    
    # Check for missing values
    print(f"Missing values before preprocessing: {combined_df.isnull().sum().sum()}")
    
    # Fill missing values - use mean for numeric columns
    numeric_columns = combined_df.select_dtypes(include=['float64', 'int64']).columns
    combined_df[numeric_columns] = combined_df[numeric_columns].fillna(combined_df[numeric_columns].mean())
    
    # Drop any rows that still have missing values
    combined_df = combined_df.dropna()
    
    # Check for imbalance in the target variable
    print("Class distribution:")
    print(combined_df['AgingRelatedBugs'].value_counts())
    print(f"Imbalance ratio: {combined_df['AgingRelatedBugs'].value_counts()[0] / combined_df['AgingRelatedBugs'].value_counts()[1]}")
    
    # Prepare features and target
    X = combined_df.drop(['Filename', 'AgingRelatedBugs'], axis=1)
    y = combined_df['AgingRelatedBugs']
    
    # Feature scaling 
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    return X_scaled, y, X.columns

# Load the data
X, y, feature_names = load_and_preprocess_data()

if DEV_MODE:
    def sample_dataset(X, y, sample_size=0.3):
        """Sample dataset while ensuring minimum number of minority samples"""
        # Get indices of minority and majority classes
        minority_indices = np.where(y == 1)[0]
        majority_indices = np.where(y == 0)[0]
        
        # Keep all minority samples to ensure SMOTE works
        selected_minority = minority_indices
        
        # Sample from majority class to reach desired ratio
        majority_sample_size = int(sample_size * len(X)) - len(minority_indices)
        majority_sample_size = max(0, majority_sample_size)  # Ensure non-negative
        
        selected_majority = np.random.choice(
            majority_indices, 
            size=min(majority_sample_size, len(majority_indices)), 
            replace=False
        )
        
        # Combine indices
        selected_indices = np.concatenate([selected_minority, selected_majority])
        
        if isinstance(X, np.ndarray):
            return X[selected_indices], y[selected_indices]
        else:
            return X.iloc[selected_indices], y.iloc[selected_indices]
    
    # Use the function
    X, y = sample_dataset(X, y, sample_size=0.3)
    print(f"Using sampled dataset: {len(X)} samples")

Missing values before preprocessing: 0
Class distribution:
AgingRelatedBugs
0    3380
1      20
Name: count, dtype: int64
Imbalance ratio: 169.0


In [70]:
def create_data_splits(X, y):
    # Split data into training, dynamic selection, and testing sets (33%, 33%, 33%)
    X_train_temp, X_test, y_train_temp, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)
    X_train, X_dsel, y_train, y_dsel = train_test_split(X_train_temp, y_train_temp, test_size=0.5, random_state=42, stratify=y_train_temp)
    
    # Count minority samples
    minority_count = Counter(y_train)[1]
    
    # Choose appropriate k for SMOTE based on minority class size
    k_neighbors = min(5, minority_count - 1)  # Must be less than minority count
    
    if k_neighbors > 0:
        # Apply SMOTE with adjusted k_neighbors
        smote = SMOTE(random_state=42, k_neighbors=k_neighbors)
        X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)
    else:
        # If too few samples, duplicate existing minority samples
        print("Too few minority samples for SMOTE. Using simple duplication.")
        X_train_balanced, y_train_balanced = simple_duplicate_balance(X_train, y_train)
    
    print("Original training set class distribution:", Counter(y_train))
    print("Balanced training set class distribution:", Counter(y_train_balanced))
    
    return X_train, y_train, X_train_balanced, y_train_balanced, X_dsel, y_dsel, X_test, y_test

def simple_duplicate_balance(X, y):
    """Balance dataset by duplicating minority class samples"""
    majority_count = Counter(y).most_common(1)[0][1]
    minority_class = Counter(y).most_common()[-1][0]
    
    minority_indices = np.where(y == minority_class)[0]
    
    # How many times to duplicate each minority sample
    n_duplicates = int(np.ceil(majority_count / len(minority_indices)))
    
    # Create duplicated minority samples
    X_minority = X[minority_indices]
    y_minority = y[minority_indices]
    
    X_minority_dup = np.repeat(X_minority, n_duplicates, axis=0)[:majority_count]
    y_minority_dup = np.repeat(y_minority, n_duplicates)[:majority_count]
    
    # Combine with majority class
    majority_indices = np.where(y != minority_class)[0]
    X_balanced = np.vstack([X[majority_indices], X_minority_dup])
    y_balanced = np.concatenate([y[majority_indices], y_minority_dup])
    
    return X_balanced, y_balanced

In [71]:
# Function to create a candidate classifier pool (Phase 1: generation phase)
def create_classifier_pool(X_train_balanced, y_train_balanced, M=100):
    """
    Creates a pool of M diverse classifiers using RandomForestClassifier with different parameters
    """
    classifier_pool = []
    
    # Create base classifiers with different parameters for diversity
    for i in range(M):
        # Vary parameters for diversity
        max_depth = np.random.randint(3, 20) if i % 3 == 0 else None
        min_samples_split = np.random.randint(2, 20) if i % 5 == 0 else 2
        min_samples_leaf = np.random.randint(1, 10) if i % 7 == 0 else 1
        
        # Create a random forest classifier with varied parameters
        clf = RandomForestClassifier(
            n_estimators=20 if DEV_MODE else 100,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bool(i % 2),
            random_state=i
        )
        
        # Train the classifier on the balanced training set
        clf.fit(X_train_balanced, y_train_balanced)
        classifier_pool.append(clf)
    
    return classifier_pool

In [72]:
# Replacing the find_k_nearest_neighbors function
def find_k_nearest_neighbors(x, dataset, K):
    """Find K nearest neighbors using scikit-learn's optimized implementation"""
    if isinstance(dataset, pd.DataFrame):
        dataset_array = dataset.values
    else:
        dataset_array = dataset
        
    # Reshape x if needed
    x_reshaped = x.reshape(1, -1) if len(x.shape) == 1 else x
    
    # Use scikit-learn's optimized KNN implementation
    nbrs = NearestNeighbors(n_neighbors=min(K, len(dataset_array)), algorithm='ball_tree').fit(dataset_array)
    _, indices = nbrs.kneighbors(x_reshaped)
    
    return indices[0]  # Return as a simple array

In [73]:
# Function to calculate output profile
def compute_output_profile(x, classifier_pool):
    """Compute the output profile of an instance based on the classifier pool predictions"""
    return np.array([clf.predict([x])[0] for clf in classifier_pool])

In [74]:
# Function to find similar output profiles
def find_similar_output_profiles(x_profile, output_profiles, K_s):
    """Find indices of K_s most similar output profiles using Hamming distance"""
    hamming_distances = np.sum(output_profiles != x_profile, axis=1)
    return np.argsort(hamming_distances)[:K_s]

In [75]:
def calculate_instance_weights(region_indices, y, alpha=0.9):
    """Calculate weights for instances in competence regions"""
    weights = []
    
    # Convert y to numpy array if it's a pandas Series
    if isinstance(y, pd.Series):
        y_array = y.values
    else:
        y_array = y
    
    # Ensure region_indices are within bounds
    valid_indices = [idx for idx in region_indices if idx < len(y_array)]
    
    for idx in valid_indices:
        current_class = y_array[idx]
        # Count instances with the same class
        num_same_class = sum(1 for i in valid_indices if y_array[i] == current_class)
        # Calculate weight
        weight = 1 / (1.0 + np.exp(alpha * num_same_class))
        weights.append(weight)
    
    # Handle empty weights list
    if not weights:
        return np.array([1.0])
    
    # Normalize weights
    weights = np.array(weights)
    normalized_weights = weights / np.sum(weights)
    
    return normalized_weights

In [76]:
def extract_meta_features(theta_j, phi_j, theta_weights, phi_weights, y, classifier, X):
    """
    Extract meta-features for a classifier based on competence regions
    
    Returns:
    - Feature vector containing 7 sets of meta-features described in the META-DESKNN-MI algorithm
    """
    meta_features = []
    
    # Convert to numpy arrays for consistent indexing
    if isinstance(X, pd.DataFrame):
        X_array = X.values
    else:
        X_array = X
        
    if isinstance(y, pd.Series):
        y_array = y.values
    else:
        y_array = y
    
    # Ensure indices are within bounds
    max_idx = len(X_array) - 1
    valid_theta_j = [idx for idx in theta_j if idx <= max_idx]
    valid_phi_j = [idx for idx in phi_j if idx <= max_idx]
    
    # Handle empty regions
    if not valid_theta_j:
        valid_theta_j = [0]  # Use first sample as fallback
    
    if not valid_phi_j:
        valid_phi_j = [0]  # Use first sample as fallback
    
    # f1: Neighbors' hard classification (based on feature space)
    f1 = []
    for idx in valid_theta_j:
        # If classifier correctly predicts the instance
        if classifier.predict([X_array[idx]])[0] == y_array[idx]:
            f1.append(1)
        else:
            f1.append(0)
    
    # f2: Posterior probability
    f2 = []
    for idx in valid_theta_j:
        # Get probability of the true class
        probs = classifier.predict_proba([X_array[idx]])[0]
        true_class_idx = int(y_array[idx])
        if true_class_idx < len(probs):  # Ensure index is valid
            f2.append(probs[true_class_idx])
        else:
            f2.append(0.0)
    
    # f3: Overall local accuracy (feature space)
    f3 = [np.mean(f1)]
    
    # f4: Output profiles classification (decision space)
    f4 = []
    for idx in valid_phi_j:
        # If classifier correctly predicts the instance
        if classifier.predict([X_array[idx]])[0] == y_array[idx]:
            f4.append(1)
        else:
            f4.append(0)
    
    # f5: Classifier's confidence (implementation simplified)
    # Using mean probability as a confidence measure
    instance_idx = valid_theta_j[0]  # Using the first neighbor to compute confidence
    probs = classifier.predict_proba([X_array[instance_idx]])[0]
    f5 = [np.max(probs)]
    
    # Adjust weights if necessary to match valid indices
    if len(theta_weights) > len(valid_theta_j):
        theta_weights = theta_weights[:len(valid_theta_j)]
    elif len(theta_weights) < len(valid_theta_j):
        # Extend weights with zeros
        theta_weights = np.pad(theta_weights, (0, len(valid_theta_j) - len(theta_weights)))
    
    if len(phi_weights) > len(valid_phi_j):
        phi_weights = phi_weights[:len(valid_phi_j)]
    elif len(phi_weights) < len(valid_phi_j):
        # Extend weights with zeros
        phi_weights = np.pad(phi_weights, (0, len(valid_phi_j) - len(phi_weights)))
    
    # f6: Overall weighted accuracy based on feature space
    f6 = [np.sum(np.array(f1) * theta_weights)]
    
    # f7: Overall weighted accuracy based on decision space
    f7 = [np.sum(np.array(f4) * phi_weights)]
    
    # Combine all meta-features
    meta_features = f1 + f2 + f3 + f4 + f5 + f6 + f7
    
    return np.array(meta_features)

In [77]:
def meta_training_phase(X_train, y_train, classifier_pool, K=7 if not DEV_MODE else 5, K_s=5 if not DEV_MODE else 3, alpha=0.9, h_c=0.8):
    """
    Implement the meta-training phase to train the meta-classifier
    
    Returns:
    - Trained meta-classifier lambda
    """
    # Convert to numpy arrays for consistent indexing
    if isinstance(X_train, pd.DataFrame):
        X_array = X_train.values
    else:
        X_array = X_train.copy()
        
    if isinstance(y_train, pd.Series):
        y_array = y_train.values
    else:
        y_array = y_train.copy()
    
    # Step 1: Sample selection process
    selected_indices = []
    y_pred_per_classifier = np.zeros((len(X_array), len(classifier_pool)))
    
    # Calculate predictions for all samples and classifiers
    for i, clf in enumerate(classifier_pool):
        y_pred_per_classifier[:, i] = clf.predict(X_array)
    
    # Check consensus of the pool for each instance
    for j in range(len(X_array)):
        # Count votes for each class
        votes = Counter(y_pred_per_classifier[j, :])
        
        # If there are at least two classes with votes
        if len(votes) >= 2:
            sorted_votes = votes.most_common()
            # Calculate difference between votes of winning class and second class
            consensus = sorted_votes[0][1] - sorted_votes[1][1]
            consensus_ratio = consensus / len(classifier_pool)
            
            # If consensus is low, select the instance
            if consensus_ratio < h_c:
                selected_indices.append(j)
    
    # If no instances selected or too few, use a minimum number
    min_required = max(K, K_s + 1)
    if len(selected_indices) < min_required:
        print(f"Too few instances selected ({len(selected_indices)}). Using top {min_required} instances.")
        # Ensure we don't select more indices than available
        max_possible = min(min_required, len(X_array))
        selected_indices = list(range(max_possible))
    
    # Also ensure that any index used later is valid
    selected_indices = [idx for idx in selected_indices if idx < len(X_array)]
    
    # Step 2: Meta-feature extraction
    X_meta = []  # Meta-features
    y_meta = []  # Meta-labels
    
    # Compute output profiles for all instances
    output_profiles = np.array([
        [clf.predict([X_array[i]])[0] for clf in classifier_pool]
        for i in range(len(X_array))
    ])
    
    for idx in selected_indices:
        # Find the competence region (K nearest neighbors)
        theta_j = find_k_nearest_neighbors(X_array[idx], X_array, K)
        
        # Compute the output profile
        output_profile = output_profiles[idx]
        
        # Find similar output profiles
        phi_j = find_similar_output_profiles(output_profile, output_profiles, K_s)
        
        # Ensure all indices are valid
        theta_j = [i for i in theta_j if i < len(X_array)]
        phi_j = [i for i in phi_j if i < len(X_array)]
        
        # If we lost all indices, use some fallback
        if not theta_j:
            theta_j = [0]  # Use first sample as fallback
        
        if not phi_j:
            phi_j = [0]  # Use first sample as fallback
        
        # Calculate instance weights
        theta_weights = calculate_instance_weights(theta_j, y_array, alpha)
        phi_weights = calculate_instance_weights(phi_j, y_array, alpha)
        
        # For each classifier, extract meta-features
        for i, clf in enumerate(classifier_pool):
            # Extract meta-features
            meta_features = extract_meta_features(theta_j, phi_j, theta_weights, phi_weights, y_array, clf, X_array)
            
            # Determine if the classifier correctly classifies the instance
            is_correct = int(clf.predict([X_array[idx]])[0] == y_array[idx])
            
            # Add meta-features and meta-label
            X_meta.append(meta_features)
            y_meta.append(is_correct)
    
    # Step 3: Train meta-classifier
    # Ensure we have non-negative values for MultinomialNB
    X_meta = np.abs(X_meta)
    
    # Handle case where all meta-labels are the same
    unique_labels = np.unique(y_meta)
    if len(unique_labels) < 2:
        print("Warning: All meta-labels are the same. Adding artificial diversity.")
        # Add some artificial diversity
        if 1 in unique_labels:
            X_meta = np.vstack([X_meta, X_meta[0]])
            y_meta = np.append(y_meta, 0)
        else:
            X_meta = np.vstack([X_meta, X_meta[0]])
            y_meta = np.append(y_meta, 1)
    
    meta_classifier = MultinomialNB()
    meta_classifier.fit(X_meta, y_meta)
    
    return meta_classifier

In [78]:
# Phase 3: Ensemble selection phase
def ensemble_selection_phase(x_test, X_dsel, y_dsel, classifier_pool, meta_classifier, K=7 if not DEV_MODE else 5, K_s=5 if not DEV_MODE else 3, alpha=0.9, J=30 if not DEV_MODE else 20):
    """
    Implement the ensemble selection phase to select the best classifiers for test instance
    
    Returns:
    - Selected classifier ensemble for the test instance
    """
    if DEV_MODE:
        # Just select the first J classifiers
        J_actual = min(J, len(classifier_pool))
        return classifier_pool[:J_actual]
    
    # Step 1: Select competent classifiers
    competent_classifiers = []
    
    # Find the competence region of the test instance
    theta_t = find_k_nearest_neighbors(x_test, X_dsel, K)
    
    # Compute output profiles for all dynamic selection instances
    output_profiles = np.array([
        [clf.predict([X_dsel[i]])[0] for clf in classifier_pool]
        for i in range(len(X_dsel))
    ])
    
    # Compute output profile for test instance
    test_output_profile = np.array([clf.predict([x_test])[0] for clf in classifier_pool])
    
    # Find similar output profiles
    phi_t = find_similar_output_profiles(test_output_profile, output_profiles, K_s)
    
    # Calculate instance weights
    theta_weights = calculate_instance_weights(theta_t, y_dsel, alpha)
    phi_weights = calculate_instance_weights(phi_t, y_dsel, alpha)
    
    # Calculate competence of each classifier
    competent_classifiers = []
    classifier_competence = []
    
    for i, clf in enumerate(classifier_pool):
        # Extract meta-features
        meta_features = extract_meta_features(theta_t, phi_t, theta_weights, phi_weights, y_dsel, clf, X_dsel)
        
        # Get competence from meta-classifier
        competence = meta_classifier.predict_proba(np.abs([meta_features]))[0][1]  # Probability of being competent
        
        # If competence is above 0.5, add to competent classifiers
        if competence > 0.5:
            competent_classifiers.append(i)
            classifier_competence.append(competence)
    
    # If no competent classifiers found, use the top J most competent from the pool
    if not competent_classifiers:
        print("No competent classifiers found. Using top J most competent classifiers.")
        all_competences = []
        for i, clf in enumerate(classifier_pool):
            meta_features = extract_meta_features(theta_t, phi_t, theta_weights, phi_weights, y_dsel, clf, X_dsel)
            competence = meta_classifier.predict_proba(np.abs([meta_features]))[0][1]
            all_competences.append((i, competence))
        
        # Sort by competence
        all_competences.sort(key=lambda x: x[1], reverse=True)
        competent_classifiers = [idx for idx, _ in all_competences[:J]]
        
    # Step 2: Select the J most diverse classifiers
    if len(competent_classifiers) <= J:
        selected_classifiers = competent_classifiers
    else:
        # Compute pairwise diversity (DF measure) between competent classifiers
        n_competent = len(competent_classifiers)
        diversity_matrix = np.zeros((n_competent, n_competent))
        
        # For all pairs of competent classifiers
        for i in range(n_competent):
            for j in range(i+1, n_competent):
                clf_i_idx = competent_classifiers[i]
                clf_j_idx = competent_classifiers[j]
                
                clf_i = classifier_pool[clf_i_idx]
                clf_j = classifier_pool[clf_j_idx]
                
                # Compute N00, N11, N01, N10 over dynamic selection set
                y_pred_i = clf_i.predict(X_dsel)
                y_pred_j = clf_j.predict(X_dsel)
                
                N00 = sum(1 for k in range(len(y_dsel)) if y_pred_i[k] != y_dsel[k] and y_pred_j[k] != y_dsel[k])
                N11 = sum(1 for k in range(len(y_dsel)) if y_pred_i[k] == y_dsel[k] and y_pred_j[k] == y_dsel[k])
                N01 = sum(1 for k in range(len(y_dsel)) if y_pred_i[k] != y_dsel[k] and y_pred_j[k] == y_dsel[k])
                N10 = sum(1 for k in range(len(y_dsel)) if y_pred_i[k] == y_dsel[k] and y_pred_j[k] != y_dsel[k])
                
                # Calculate DF diversity
                if (N11 + N01 + N10 + N00) > 0:  # Avoid division by zero
                    df_ij = N00 / (N11 + N01 + N10 + N00)
                else:
                    df_ij = 0
                
                diversity_matrix[i, j] = df_ij
                diversity_matrix[j, i] = df_ij
        
        # Calculate overall diversity of each classifier
        overall_diversity = np.sum(diversity_matrix, axis=1)
        
        # Select the J classifiers with highest diversity
        selected_indices = np.argsort(overall_diversity)[-J:]
        selected_classifiers = [competent_classifiers[i] for i in selected_indices]
    
    # Return the selected ensemble
    return [classifier_pool[i] for i in selected_classifiers]

In [79]:
# Phase 4: Combination phase
def predict_with_ensemble(selected_ensemble, x_test):
    """
    Combine predictions of selected classifiers using majority voting
    
    Returns:
    - Final prediction
    """
    predictions = [clf.predict([x_test])[0] for clf in selected_ensemble]
    
    # Apply majority voting
    prediction_counts = Counter(predictions)
    final_prediction = prediction_counts.most_common(1)[0][0]
    
    return final_prediction

In [80]:
def meta_desknn_mi(X_train, y_train, X_train_balanced, y_train_balanced, X_dsel, y_dsel, X_test, y_test):
    # Phase 1: Generation phase - Create classifier pool
    print("Phase 1: Generation phase - Creating classifier pool...")
    classifier_pool = create_classifier_pool(X_train_balanced, y_train_balanced, M=20 if DEV_MODE else 100)
    print(f"Created pool of {len(classifier_pool)} classifiers")
    
    # Phase 2: Meta-training phase - Train meta-classifier
    print("Phase 2: Meta-training phase - Training meta-classifier...")
    meta_classifier = meta_training_phase(X_train, y_train, classifier_pool)
    print("Meta-classifier trained")
    
    # Phase 3 & 4: For each test instance, select ensemble and predict
    print("Phase 3 & 4: Ensemble selection and combination for test set...")
    y_pred = []
    y_prob = []
    
    # Use smaller test set in dev mode
    test_instances = X_test[:100] if DEV_MODE and len(X_test) > 100 else X_test
    
    for i, x_test in enumerate(test_instances):
        if i % 100 == 0:
            print(f"Processing test instance {i}/{len(test_instances)}")
        
        # Phase 3: Ensemble selection phase
        selected_ensemble = ensemble_selection_phase(
            x_test, X_dsel, y_dsel, classifier_pool, meta_classifier
        )
        
        # Phase 4: Combination phase - predict using majority voting
        prediction = predict_with_ensemble(selected_ensemble, x_test)
        y_pred.append(prediction)
        
        # Calculate probabilities for ROC AUC
        probs = [clf.predict_proba([x_test])[0][1] for clf in selected_ensemble]
        avg_prob = np.mean(probs)
        y_prob.append(avg_prob)
    
    # Evaluate results
    if len(y_pred) < len(y_test):
        # We only processed a subset in dev mode, so use the same subset for evaluation
        test_y = y_test[:len(y_pred)]
    else:
        test_y = y_test
        
    print("\nResults:")
    print(classification_report(test_y, y_pred))
    print("\nConfusion Matrix:")
    print(confusion_matrix(test_y, y_pred))
    
    # Calculate AUC
    auc = roc_auc_score(test_y, y_prob)
    print(f"\nAUC: {auc:.4f}")
    
    return y_pred, y_prob, auc

In [81]:
# Main execution
if __name__ == "__main__":
    print("Loading and preprocessing data...")
    X, y, feature_names = load_and_preprocess_data()
    
    print("\nCreating data splits...")
    X_train, y_train, X_train_balanced, y_train_balanced, X_dsel, y_dsel, X_test, y_test = create_data_splits(X, y)
    
    print("\nRunning META-DESKNN-MI algorithm...")
    y_pred, y_prob, auc = meta_desknn_mi(X_train, y_train, X_train_balanced, y_train_balanced, X_dsel, y_dsel, X_test, y_test)
    
    print("\nDone!")

Loading and preprocessing data...
Missing values before preprocessing: 0
Class distribution:
AgingRelatedBugs
0    3380
1      20
Name: count, dtype: int64
Imbalance ratio: 169.0

Creating data splits...
Original training set class distribution: Counter({0: 1132, 1: 7})
Balanced training set class distribution: Counter({1: 1132, 0: 1132})

Running META-DESKNN-MI algorithm...
Phase 1: Generation phase - Creating classifier pool...
Created pool of 100 classifiers
Phase 2: Meta-training phase - Training meta-classifier...
Meta-classifier trained
Phase 3 & 4: Ensemble selection and combination for test set...
Processing test instance 0/1122


KeyError: 0

In [ ]:
# Run the META-DESKNN-MI model
print("Running META-DESKNN-MI algorithm...")
y_pred, y_prob, auc = meta_desknn_mi(X_train, y_train, X_train_balanced, y_train_balanced, X_dsel, y_dsel, X_test, y_test)

# Create visualizations
plt.figure(figsize=(16, 12))

# In dev mode, we need to use only the subset of test data that was processed
if DEV_MODE and len(y_pred) < len(y_test):
    # Use the same subset for visualization that was used for prediction
    test_y = y_test[:len(y_pred)]
else:
    test_y = y_test

# ROC Curve
plt.subplot(2, 2, 1)
from sklearn.metrics import roc_curve
fpr, tpr, _ = roc_curve(test_y, y_prob)
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (area = {auc:.4f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")

# Confusion Matrix
plt.subplot(2, 2, 2)
cm = confusion_matrix(test_y, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')

plt.tight_layout()
plt.show()

print(f"\nFinal AUC: {auc:.4f}")
print("\nClassification Report:")
print(classification_report(test_y, y_pred))